# Get Reserve Info

In this notebook, we gather information regarding the types of tokens present in Aave.

In [45]:
#import packages
import requests
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import matplotlib.dates as md
import math
import time

## Get Rate Data

If this data was previously compiled, we load the existing data.

In [46]:
start_time = 0
try:
    old_data = pd.read_csv('rates.csv')
    exists = True
    start_time = old_data['timestamp'].max()
    old_data.info()
except:
    old_data = pd.DataFrame()
    exists = False

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 689814 entries, 0 to 689813
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   liquidityRate       689814 non-null  float64
 1   reserve             689814 non-null  object 
 2   stableBorrowRate    689814 non-null  float64
 3   timestamp           689814 non-null  int64  
 4   variableBorrowRate  689814 non-null  float64
 5   datetime            689814 non-null  object 
dtypes: float64(3), int64(1), object(2)
memory usage: 31.6+ MB


We start by gathering data for borrow and deposit rates for each currency. The data has a row for every time a rate was updated for a given currency.

In [47]:
token_data=[]
#loop until no more data left
while(1):
    try:
        #set query
        query="""
        {
            reserveParamsHistoryItems(first: 1000 orderBy: timestamp where:{timestamp_gt:"""+str(start_time)+"""}) {
            timestamp
            reserve{
                symbol
            }
            variableBorrowRate
            stableBorrowRate
            liquidityRate
            }
        }
        """
        #make request
        url = 'https://api.thegraph.com/subgraphs/name/aave/protocol-v2'
        request = requests.post(url,json={'query':query})
        #store data
        token_data.extend(request.json()['data']['reserveParamsHistoryItems'])
        start_time = str(request.json()['data']['reserveParamsHistoryItems'][-1]['timestamp'])
    except Exception as e:
        #exit when no more data left to get
        print(e)
        break

#create borrows data frame
df_tokens = pd.DataFrame(token_data)
df_tokens.info()

list index out of range
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Empty DataFrame

In [48]:
df_tokens.head()

""


### Cleaning

We adjust the values for the rates by a scaler to represent these as percentages.

In [49]:
#adjust rates to percentages
df_tokens['liquidityRate']= df_tokens['liquidityRate'].astype(float)/1e27*100
df_tokens['variableBorrowRate']= df_tokens['variableBorrowRate'].astype(float)/1e27*100
df_tokens['stableBorrowRate']= df_tokens['stableBorrowRate'].astype(float)/1e27*100

df_tokens[['liquidityRate','variableBorrowRate','stableBorrowRate']]

KeyError: 'liquidityRate'

Next, we transform the reserve column to string.

In [ ]:
#define function to parse reserves
def getReserves(row):
    reserve = row['reserve']['symbol']
    return reserve
 
#apply to reserve column to convert to string
df_tokens['reserve'] = df_tokens.apply(lambda x: getReserves(x), axis=1)

In [ ]:
df_tokens['datetime']=pd.to_datetime(df_tokens['timestamp'],unit='s')

Finally, we save the data to a csv file.

In [ ]:
#save to csv file
df_tokens = old_data.append(df_tokens)
df_tokens.to_csv("rates.csv",index=False)
df_tokens.info()

### Analysis

We will perform some basis analysis on CRV data. We subset the dataset so only rows regarding CSV are included.

In [ ]:
#subset crv
df_crv = df_tokens[df_tokens['reserve']=='CRV']
df_crv.info()

We create a boxplot to show the distribution of liquidity rates for CSV.

In [ ]:
#create boxplot
sns.boxplot(x=df_crv['liquidityRate'].astype(float))
plt.show()

### Get Token Data

Next, we will create a dataset where each row is a unique currency, and the columns are different metrics to describe it.

In [ ]:
#set query
query="""
        {
            reserves{
                id
                pool{
                    id
                }
                symbol
                decimals
                usageAsCollateralEnabled
                borrowingEnabled
                stableBorrowRateEnabled
                reserveLiquidationThreshold
                reserveLiquidationBonus
            }
        }
        """

#make request
url = 'https://api.thegraph.com/subgraphs/name/aave/protocol-v2'
request = requests.post(url,json={'query':query})

#save result as dataframe
df_token_data = pd.DataFrame(request.json()['data']['reserves'])
df_token_data.info()

### Cleaning

First, we convert the pool column to string.

In [ ]:
#function to parse pool
def getPool(row):
    pool = row['pool']['id']
    return pool

#apply to convert pool column to string
df_token_data['pool'] = df_token_data.apply(lambda x: getPool(x), axis=1)

Next, we convert the numeric columns to floats.

In [ ]:
#change numeric columns to type float
df_token_data['reserveLiquidationBonus']=df_token_data['reserveLiquidationBonus'].astype(float)
df_token_data['reserveLiquidationThreshold']=df_token_data['reserveLiquidationThreshold'].astype(float)

In [ ]:
df_token_data.head()

In [ ]:
df_token_data['symbol']

Finally, we save the data as a csv file.

In [ ]:
#save to csv file
df_token_data.to_csv("tokens.csv",index=False)

### Analysis

Below, we create a bar chart to show the number of coins that have borrowing enabled. The majority of coins have borrowing enabled.

In [ ]:
#get value counts for columns
metric='borrowingEnabled'
stats = df_token_data[metric].value_counts()

#create barplot
sns.barplot(x=stats.index,y=stats.values)
plt.title(metric)
plt.show()

Next, we examine which coins can be used as collateral. The vast majority coins are available to be used as collateral.

In [ ]:
#get value counts for columns
metric='usageAsCollateralEnabled'
stats = df_token_data[metric].value_counts()

#create barplot
sns.barplot(x=stats.index,y=stats.values)
plt.title(metric)
plt.show()

Now, we examine the number of coins that have stable borrow rates enabled. We can see that most coins do not have stable borrow rates enabled.

In [ ]:
#get value counts for columns
metric='stableBorrowRateEnabled'
stats = df_token_data[metric].value_counts()

#create barplot
sns.barplot(x=stats.index,y=stats.values)
plt.title(metric)
plt.show()

We create a swarmplot to show the distribution of reserve liquidation bonuses. It seems that this column is not continous. Many coins have a bonus of 0, whereas the rest of coins have a bonus ranging from 10,000 - 12,000.

In [ ]:
#create swarmplot
sns.swarmplot(data=df_token_data,x='reserveLiquidationBonus')
plt.title('reserveLiquidationBonus')
plt.show()

In [ ]:
df_token_data

Finally, we analyze the reserve liquidation threshold for coins. Many have a threshold of 0, where the rest have a threshold centered around 7,000.

In [ ]:
#create swarmplot
sns.swarmplot(data=df_token_data,x='reserveLiquidationThreshold')
plt.title('reserveLiquidationThreshold')
plt.show()